# Setup Azure Kubernetes Infrastructure
In this notebook we will setup 
- An AKS cluster with
  - **GPU enabled Spot VM nodepool** for running elastic training
  - **CPU VM nodepool** for running Rendezevous server - training control plane
- Azure Storage Account for hosting training data and model training checkpoints
- Deploy Kubernetes Components
  - **Azure Blob CSI Driver** to map Blob storage to container as persistent volumes
  - Kubernetes **PersistentVolume** and PersistentVolumeClaim

## Define Variables
Set variables required for the project

In [ ]:
subscription_id = ""             # fill in
resource_group = "elastic-lab"   # feel free to replace or use this default
region = "eastus2"               # ffeel free to replace or use this default

storage_account_name = "<name >"        # fill in
storage_container_name = "workerdata"             

aks_name = "elasticaks"    # feel free to replace or use this default
aks_spot_nodepool = "spotgpu"       # feel free to replace or use this default
aks_cpu_nodepool = "cpuworkers"     # feel free to replace or use this default
aks_gpu_sku = "Standard_NC12"       # feel free to replace or use this default 

## Azure account login
If you are not already logged in to an Azure account, the command below will initiate a login. This will pop up a browser where you can select your login. (if no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code` or login in WSL command  prompt and proceed to notebook)

In [ ]:
%%bash
az login -o table


In [ ]:
!az account set --subscription "$subscription_id"

In [ ]:
!az account show

## Create Resource Group
Azure encourages the use of groups to organize all the Azure components you deploy. That way it is easier to find them but also we can delete a number of resources simply by deleting the group.

In [ ]:
!az group create -l {region} -n {resource_group}

## Create AKS Cluster and NodePools
Below, we create the AKS cluster with default 1 system node (to save time, in production use more nodes as per best practices) in the resource group we created earlier. This step can take 5 or more minutes.


In [ ]:
%%time
!az aks create --resource-group {resource_group} \
    --name {aks_name} \
    --node-vm-size Standard_D2s_v3 \
    --node-count 1 \
    --location {region}  \
    --kubernetes-version 1.18.17 \
    --generate-ssh-keys

## Connect to AKS Cluster
To configure kubectl to connect to Kubernetes cluster, run the following command

In [ ]:
!az aks get-credentials --resource-group {resource_group} --name {aks_name}

Let's verify connection by listing the nodes.

In [ ]:
!kubectl get nodes

Taint System node with `CriticalAddonsOnly` taint so it is available only for system workloads

In [ ]:
!kubectl taint nodes -l agentpool=nodepool1 CriticalAddonsOnly=true:NoSchedule --overwrite


## Create GPU enabled and CPU Node Pools
To create GPU enabled nodepool, will use fully configured AKS image that contains the NVIDIA device plugin for Kubenetes, see [Use the AKS specialized GPU image (preview)](https://docs.microsoft.com/en-us/azure/aks/gpu-cluster#use-the-aks-specialized-gpu-image-preview). Creating nodepools could take five or more minutes.

In [ ]:
%%time
!az feature register --name GPUDedicatedVHDPreview --namespace Microsoft.ContainerService
!az feature list -o table --query "[?contains(name, 'Microsoft.ContainerService/GPUDedicatedVHDPreview')].{Name:name,State:properties.state}"
!az provider register --namespace Microsoft.ContainerService
!az extension add --name aks-preview


For more information on Azure Spot Nodepool https://docs.microsoft.com/en-us/azure/aks/spot-node-pool

In [ ]:
%%time
!az aks nodepool add \
    --resource-group {resource_group} \
    --cluster-name {aks_name} \
    --name {aks_spot_nodepool} \
    --priority Spot \
    --eviction-policy Delete \
    --spot-max-price -1 \
    --node-count 2 \
    --node-vm-size {aks_gpu_sku} \
    --aks-custom-headers UseGPUDedicatedVHD=true,usegen2vm=true

## Verify GPU is available on Kubernetes Node
Now use the kubectl describe node command to confirm that the GPUs are schedulable. Under the Capacity section, the GPU should list as nvidia.com/gpu: 2.

In [ ]:
!kubectl describe node -l kubernetes.azure.com/scalesetpriority=spot

## Create CPU NodePool for running ETCD

In [ ]:
%%time 
!az aks nodepool add \
  --resource-group {resource_group} \
    --cluster-name {aks_name} \
    --name {aks_cpu_nodepool} \
    --enable-cluster-autoscaler \
    --min-count 1 \
    --max-count 3 \
    --node-vm-size Standard_D2s_v3 

## Verify Taints on the Kubernetes nodes
Verify that system pool and have the Taints `CriticalAddonsOnly` and `kubernetes.azure.com/scalesetpriority` respectively   


In [ ]:
!kubectl get nodes -o json | jq '.items[].spec.taints'

# Create Storage Account for training data 
In this section of the notebook, we'll create an Azure blob storage that we'll use throughout the tutorial. This object store will be used to store input images and save checkpoints. Use `az cli` to create the account

In [ ]:
%%time
!az storage account create -n {storage_account_name} -g {resource_group} --query 'provisioningState'


Grab the keys of the storage account that was just created.We would need them for binding Kubernetes Persistent Volume. The --quote '[0].value' part of the command simply means to select the value of the zero-th indexed of the set of keys.

In [ ]:
key = !az storage account keys list --account-name {storage_account_name} -g {resource_group} --query '[0].value'


The stdout from the command above is stored in a string array of 1. Select the element in the array and strip opening and closing quotation marks.

In [ ]:
storage_account_key = str(key[0][1:-1]) # this is used to strip opening and closing quotation marks

In [ ]:
# create storage container

!az storage container create \
    --account-name {storage_account_name} \
    --account-key {storage_account_key} \
    --name {storage_container_name}

# Install Kubernetes Blob CSI Driver 
[Azure Blob Storage CSI driver for Kubernetes](https://github.com/kubernetes-sigs/blob-csi-driver) allows Kubernetes to access Azure Storage. We will deploy it using Helm3 package manager as described in the docs https://github.com/kubernetes-sigs/blob-csi-driver/tree/master/charts

In [ ]:
!helm repo add blob-csi-driver https://raw.githubusercontent.com/kubernetes-sigs/blob-csi-driver/master/charts
!helm install blob-csi-driver blob-csi-driver/blob-csi-driver --namespace kube-system --version v1.1.0



In [ ]:
!kubectl -n kube-system get pods -l "app.kubernetes.io/instance=blob-csi-driver"

## Create Persistent Volume for Azure Blob
For more details on creating   `PersistentVolume` using CSI driver refer to https://github.com/kubernetes-sigs/blob-csi-driver/blob/master/deploy/example/e2e_usage.md

In [ ]:
!kubectl create namespace elastic-job
# Create secret to access storage account
!kubectl create secret generic azure-blobsecret --from-literal azurestorageaccountname={storage_account_name} --from-literal azurestorageaccountkey="{storage_account_key}" --type=Opaque -n elastic-job

Persistent Volume YAML definition is in `kube/azure-blobfules-pv.yaml` with fields pointing to secret created above and containername we created in storage account:
```yaml
  csi:
    driver: blob.csi.azure.com
    readOnly: false
    volumeHandle: trainingdata  # make sure this volumeid is unique in the cluster
    volumeAttributes:
      containerName: workerdata # Modify if changed in Notebook
    nodeStageSecretRef:
      name: azure-blobsecret
      namespace: elastic-job
```

In [ ]:
# Create PersistentVolume and PersistenVollumeClaim for container mounts
!kubectl apply -f kube/azure-blobfuse-pv.yaml

Now all the Kubernetes preparation steps are done, we will look at adjusting training script to be able to run it in Elastic Fault tolerant way 

Proceed to [Step 2 Distributed Training Script](/Step2-DistributedTraining.md)